In [52]:
# importing libraries needed
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

In [16]:
df=pd.read_csv('Clean_data.csv')

In [17]:
df.isna().sum()

Unnamed: 0                                                    0
status                                                        0
duration                                                      0
amount                                                        0
employment_duration                                           0
age                                                           0
credit_risk                                                   0
credit_history_critical account/other credits elsewhere       0
credit_history_delay in paying off in the past                0
credit_history_existing credits paid back duly till now       0
credit_history_no credits taken/all credits paid back duly    0
purpose_car (new)                                             0
purpose_car (used)                                            0
purpose_domestic appliances                                   0
purpose_furniture/equipment                                   0
purpose_others                          

In [18]:
X=df.drop(columns='credit_risk')
Y=df['credit_risk']

In [19]:
train,train_val,test,test_val=train_test_split(X,Y,test_size=0.4,random_state=7)

In [20]:
X_train,X_test,Y_train,Y_test=train_test_split(train,test,test_size=0.2,random_state=3)

# Using knn as 1 base model

In [21]:
knn=KNeighborsClassifier ()
knn.fit(X_train,Y_train)

KNeighborsClassifier()

In [22]:
knn.score(X_test,Y_test)

0.9

# Using svm as 2 base model

In [23]:
svm=SVC()
svm.fit(X_train,Y_train)

SVC()

In [24]:
svm.score(X_test,Y_test)

0.8416666666666667

Now prdiciting the values using Svm and knn model

In [25]:
Pre1=knn.predict(train_val)
Pre2=svm.predict(train_val)

Now making a stack of both base model predictions using numpy

In [26]:
Pre=np.column_stack((Pre1,Pre2))

In [27]:
predict_test1 = knn.predict(X_test)
predict_test2 = svm.predict(X_test)

In [28]:
predict_test = np.column_stack((predict_test1,predict_test2))
predict_test

array([[1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0,

# Using Randomforest as final Model

In [31]:
random= RandomForestClassifier ()
random.fit(Pre,test_val)

RandomForestClassifier()

In [32]:
random.score(predict_test,Y_test)

0.9

# Look at parameters used by our current forest

In [34]:
print('Parameters currently in use:\n')
print(random.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


# Using Random Hyperparameter Grid

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [42]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 400, num = 25)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10,50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [43]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [44]:
print(random_grid)

{'n_estimators': [200, 208, 216, 225, 233, 241, 250, 258, 266, 275, 283, 291, 300, 308, 316, 325, 333, 341, 350, 358, 366, 375, 383, 391, 400], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [45]:
rf_random = RandomizedSearchCV(estimator = random, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [46]:
rf_random.fit(Pre,test_val)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 14, 18, 23, 27, 32,
                                                      36, 41, 45, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 208, 216, 225,
                                                         233, 241, 250, 258,
                                                         266, 275, 283, 291,
                                                         300, 308, 316, 325,
                                                         333, 341, 350, 358,
                                                         366, 375, 383,

In [48]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

In [57]:
x=rf_random.score(predict_test,Y_test)

In [58]:
print("The final Score of the Model with Random hyperparameter grid is ", x )

The final Score of the Model with Random hyperparameter grid is  0.8916666666666667


# save the model to disk


In [55]:
filename = 'finalized_model.sav'
pickle.dump(rf_random, open(filename, 'wb'))